In [2]:
"""
Test script for the SQL agent with various queries.
"""

import sys
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from backend.core.agent import SchemaAwareAgent
from backend.database.connection import get_database_engine
from backend.core.schema_manager import SchemaManager
from sqlalchemy import inspect

def run_test(agent, query, test_name):
    """Run a test query and print results"""
    print(f"\n{'='*80}")
    print(f"Test: {test_name}")
    print(f"Query: {query}")
    try:
        result = agent.run_with_reasoning(query)
        print(result)
    except Exception as e:
        print(f"Error: {str(e)}")
    print('='*80)

def main():
    # Initialize the agent with existing database
    print("Initializing SQL Agent...")
    engine = get_database_engine()
    
    # Print available tables and their schemas
    inspector = inspect(engine)
    available_tables = inspector.get_table_names()
    print("\nAvailable Tables:")
    print(available_tables)
    
    # Initialize schema manager and load schemas
    schema_manager = SchemaManager()
    for table in available_tables:
        schema_manager.load_schema(table)
        print(f"\n{table} Table Schema:")
        print(schema_manager.get_schema(table))
    
    # Initialize agent
    agent = SchemaAwareAgent(engine)
    
    # Test cases
    test_cases = [
        {
            "name": "Simple customer purchase query",
            "query": "What items did Alice buy?"
        },
        {
            "name": "Aggregation query",
            "query": "What is the total amount spent by each customer?"
        },
        {
            "name": "Complex conditions query",
            "query": "Which customers spent more than $30 and left a tip greater than $10?"
        },
        {
            "name": "Calculation query",
            "query": "What is the average tip percentage for each customer?"
        },
        {
            "name": "Ordering and limiting query",
            "query": "What are the top 3 most expensive items purchased?"
        },
        {
            "name": "Multiple table join query",
            "query": "Show me all purchases with customer names and item details"
        },
        {
            "name": "Group by with having query",
            "query": "Which customers have made more than 2 purchases?"
        }
    ]
    
    # Run all tests
    for test in test_cases:
        run_test(agent, test["query"], test["name"])

if __name__ == "__main__":
    main() 

Initializing SQL Agent...

Available Tables:
['customers', 'items', 'receipts']

customers Table Schema:
{'columns': [{'name': 'customer_id', 'type': 'INTEGER', 'nullable': False, 'primary_key': 1}, {'name': 'name', 'type': 'VARCHAR(32)', 'nullable': True, 'primary_key': 0}, {'name': 'email', 'type': 'VARCHAR(64)', 'nullable': True, 'primary_key': 0}], 'foreign_keys': []}

items Table Schema:
{'columns': [{'name': 'item_id', 'type': 'INTEGER', 'nullable': False, 'primary_key': 1}, {'name': 'name', 'type': 'VARCHAR(32)', 'nullable': True, 'primary_key': 0}, {'name': 'price', 'type': 'FLOAT', 'nullable': True, 'primary_key': 0}], 'foreign_keys': []}

receipts Table Schema:
{'columns': [{'name': 'receipt_id', 'type': 'INTEGER', 'nullable': False, 'primary_key': 1}, {'name': 'customer_id', 'type': 'INTEGER', 'nullable': True, 'primary_key': 0}, {'name': 'item_id', 'type': 'INTEGER', 'nullable': True, 'primary_key': 0}, {'name': 'quantity', 'type': 'INTEGER', 'nullable': True, 'primary_key'

Device set to use mps:0



Test: Simple customer purchase query
Query: What items did Alice buy?

SQL Query:
SELECT items.name 
FROM items 
JOIN receipts ON items.item_id = receipts.item_id 
JOIN customers ON receipts.customer_id = customers.customer_id 
WHERE customers.name = 'Alice';

Result:
[{'name': 'Coffee'}, {'name': 'Sandwich'}]

Test: Aggregation query
Query: What is the total amount spent by each customer?

SQL Query:
SELECT customers.customer_id, customers.name, SUM(items.price * receipts.quantity) AS total_spent
FROM customers
JOIN receipts ON customers.customer_id = receipts.customer_id
JOIN items ON receipts.item_id = items.item_id
GROUP BY customers.customer_id, customers.name;

Result:
[{'customer_id': 1, 'name': 'Alice', 'total_spent': 25.5}, {'customer_id': 2, 'name': 'Bob', 'total_spent': 45.75}, {'customer_id': 3, 'name': 'Charlie', 'total_spent': 15.25}, {'customer_id': 4, 'name': 'Diana', 'total_spent': 60.0}]

Test: Complex conditions query
Query: Which customers spent more than $30 and l

2025-05-25 19:44:04,422 - backend.tools.sql_agent_tool - ERROR - SQL query execution failed: (sqlite3.OperationalError) near "I": syntax error
[SQL: I'm sorry, but based on the provided database schema, there is no information about tips or any related data. The schema only includes information about customers and items. Therefore, it's not possible to calculate the average tip percentage for each customer. Please provide the correct database schema or table that includes tip information.]
(Background on this error at: https://sqlalche.me/e/20/e3q8)



SQL Query:
I'm sorry, but based on the provided database schema, there is no information about tips or any related data. The schema only includes information about customers and items. Therefore, it's not possible to calculate the average tip percentage for each customer. Please provide the correct database schema or table that includes tip information.

Result:
Error executing query: (sqlite3.OperationalError) near "I": syntax error
[SQL: I'm sorry, but based on the provided database schema, there is no information about tips or any related data. The schema only includes information about customers and items. Therefore, it's not possible to calculate the average tip percentage for each customer. Please provide the correct database schema or table that includes tip information.]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Test: Ordering and limiting query
Query: What are the top 3 most expensive items purchased?

SQL Query:
SELECT items.name, items.price
FROM items
J